In [48]:
!pip install scikit-optimize

    100% |████████████████████████████████| 81kB 3.1MB/s 


In [2]:
!pip install comet_ml
!pip install urllib==1.34


    100% |████████████████████████████████| 92kB 3.4MB/s 
    100% |████████████████████████████████| 204kB 23.0MB/s 
    100% |████████████████████████████████| 389kB 24.2MB/s 
    100% |████████████████████████████████| 122kB 31.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj
requests 2.18.4 has requirement urllib3<1.23,>=1.21.1, but you'll have urllib3 1.24.1 which is incompatible.
fastai 1.0.49 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.22
    Uninstalling urllib3-1.22:
      Successfully uninstalled urllib3-1.22


  Could not find a version that satisfies the requirement urllib==1.34 (from versions: )
No matching distribution found for urllib==1.34


In [0]:
from comet_ml import Experiment

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
print(torch.__version__)

1.0.1.post2


## Fashion MINST dataset


In [0]:
from torchvision.datasets import FashionMNIST

In [4]:
# Getting the train and test parts of the dataset
data_train = FashionMNIST("FashionMNIST/",
                          download=True,
                          train=True)

data_test = FashionMNIST("FashionMNIST/",
                          download=True,
                          train=False)

# In fact, it's already stored as torch tensor, but we'll need
# to work with the numpy representation, so let's do the convertion:
X_train = data_train.train_data.numpy()
y_train = data_train.train_labels.numpy()

X_test = data_test.test_data.numpy()
y_test = data_test.test_labels.numpy()


0it [00:00, ?it/s]

26427392it [00:04, 5665981.02it/s]                              


Extracting FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 38315.76it/s]                           
0it [00:00, ?it/s]

...

4423680it [00:02, 1592872.26it/s]                            
0it [00:00, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 13250.32it/s]            

Extracting FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!



/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


The datasets consists of images belonging to one out of 10 classes:

| Label | Description | | Label | Description |
| ---       
| 0        | T-shirt/top   || 5        | Sandal         |
| 1        | Trouser        || 6        | Shirt             |
| 2        | Pullover       || 7        | Sneaker       |
| 3        | Dress           || 8        | Bag              |
| 4        | Coat             || 9        | Ankle boot  |






### Input preprocessing

So far our data is held as numpy arrays of unsigned byte type, i.e. it lies within a range from 0 to 255. Also, the shape of our input is 3-dimensional (num_images, height, width), while our `model` takes 2-dimensional "arrays of 1-dimensional images" (num_images, height * width).

We have to convert that to `torch` tensors and reshape the input. Also, it's a good idea to normalize your image data to lie within a $[0, 1]$ interval. Let's write a function that does all these things:

In [0]:
# Write a function to convert X and y to torch tensors while
# rescaling X to fit into [0, 1] interval and reshaping it properly

# Hint: make sure your input tensor dtype is same as the
# parameters of the model (should be torch.float)

def preprocess_data(X, y):
  X_preprocessed = torch.reshape(torch.from_numpy(X/ 255.).type(torch.float32), (-1, 784))
  y_preprocessed = torch.from_numpy(y)
  return X_preprocessed, y_preprocessed

The simplest way to check this function is to use it to calculate loss:

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/anaderi/optimize/ef1b9b9aa0b64d0fbb51b48a10f72f73



### Some utilities

In [0]:
# Batch generator
# (here's a very brief description of what python generators are:
# https://stackoverflow.com/a/231855/3801744)
def get_batches(X, y, batch_size, shuffle=False):
  if shuffle:
    shuffle_ids = np.random.permutation(len(X))
    X = X[shuffle_ids].copy()
    y = y[shuffle_ids].copy()
  for i_picture in range(0, len(X), batch_size):
    # Get batch and preprocess it:
    batch_X = X[i_picture:i_picture + batch_size]
    batch_y = y[i_picture:i_picture + batch_size]
    
    # 'return' the batch (see the link above to
    # better understand what 'yield' does)
    yield preprocess_data(batch_X, batch_y)

We'll add one more utility to keep track of loss values and plot them at each epoch:

In [0]:
from IPython.display import clear_output

class Logger:
  def __init__(self):
    self.train_loss_batch = []
    self.train_loss_epoch = []
    self.test_loss_batch = []
    self.test_loss_epoch = []
    self.train_batches_per_epoch = 0
    self.test_batches_per_epoch = 0
    self.epoch_counter = 0

  def fill_train(self, loss):
    self.train_loss_batch.append(loss)
    self.train_batches_per_epoch += 1

  def fill_test(self, loss):
    self.test_loss_batch.append(loss)
    self.test_batches_per_epoch += 1

  def finish_epoch(self):
    self.train_loss_epoch.append(np.mean(
        self.train_loss_batch[-self.train_batches_per_epoch:]
    ))
    self.test_loss_epoch.append(np.mean(
        self.test_loss_batch[-self.test_batches_per_epoch:]
    ))
    self.train_batches_per_epoch = 0
    self.test_batches_per_epoch = 0
    
    clear_output()
  
    print("epoch #{} \t train_loss: {:.8} \t test_loss: {:.8}".format(
              self.epoch_counter,
              self.train_loss_epoch[-1],
              self.test_loss_epoch [-1]
          ))
    
    self.epoch_counter += 1

    plt.figure(figsize=(11, 5))

    plt.subplot(1, 2, 1)
    plt.plot(self.train_loss_batch, label='train loss')
    plt.xlabel('# batch iteration')
    plt.ylabel('loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(self.train_loss_epoch, label='average train loss')
    plt.plot(self.test_loss_epoch , label='average test loss' )
    plt.legend()
    plt.xlabel('# epoch')
    plt.ylabel('loss')
    plt.show();

### Putting it all together

Here's the setting up part, copy-pasted from previous cells:

In [0]:
def get_test_predictions(model, batch_size=100):
  predictions_test = np.concatenate([
    model(batch_X).to('cpu').detach().numpy()
    for batch_X, batch_y in get_batches(X_test, y_test, batch_size)
  ], axis=0)
  return np.argmax(predictions_test, axis=1)

from sklearn.metrics import accuracy_score

In [0]:
def train2score(n_epochs, batch_size=100, learning_rate=0.001, hidden_size=100):
  global experiment
  experiment.log_parameters({
      'n_epochs': n_epochs,
      'batch_size': batch_size,
      'learning_rate': learning_rate,
      'hidden_size': hidden_size})
  #logger = Logger()
  # Defining the loss function:
  loss_function = torch.nn.CrossEntropyLoss()

  # Defining the model
  input_size = 28 * 28 # number of pixels per image
  output_size = 10 # number of classes
  #model = torch.nn.Linear(in_features=input_size,
  #                        out_features=output_size)

  model = torch.nn.Sequential(
    torch.nn.Linear(input_size, hidden_size),
    torch.nn.ELU(),
    torch.nn.Linear(hidden_size, output_size),
  )

  # Setting up the optimizer
  learning_rate = 0.005
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  for i_epoch in range(n_epochs):
    for batch_X, batch_y in get_batches(X_train, y_train,
                                        batch_size=batch_size,
                                        shuffle=True):

      # Compute the loss, zero the gradients, and make an optimization step
      predictions = model(batch_X) # compute the predictions
      loss = loss_function(predictions, batch_y) # compute the loss
      #logger.fill_train(loss.item())

      model.zero_grad() # zero the gradients
      loss.backward() # compute new gradients
      optimizer.step() # do an optimization step

      #logger.fill_train(loss.item())

    for batch_X, batch_y in get_batches(X_test, y_test,
                                        batch_size=batch_size):

      # Compute the loss
      predictions = model(batch_X) # compute the predictions
      loss = loss_function(predictions, batch_y) # compute the loss

      #logger.fill_test(loss.item())

    #logger.finish_epoch()
  accuracy = accuracy_score(get_test_predictions(model), y_test)
  experiment.log_metrics({'accuracy': accuracy, 'loss': loss.item()})
  experiment.end()
  return accuracy

In [0]:
experiment = Experiment(api_key="JmICwg0qOenLqzP7m5lXA1szo", project_name="optimize")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/anaderi/optimize/52aba0a221084a6daf3627462f6bef2c



In [0]:
n_epochs = 1
learning_rate = 0.0005
hidden_size = 100

score = train2score(n_epochs=n_epochs, learning_rate=learning_rate, 
                    hidden_size=hidden_size)
print ("Score: {}".format(score))

COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.ml/anaderi/optimize/52aba0a221084a6daf3627462f6bef2c
COMET INFO:   Metrics:
COMET INFO:                      accuracy: 0.842
COMET INFO:                          loss: 0.4502934217453003
COMET INFO:         sys.gpu.0.free_memory: 11985420288
COMET INFO:     sys.gpu.0.gpu_utilization: 0
COMET INFO:        sys.gpu.0.total_memory: 11996954624
COMET INFO:         sys.gpu.0.used_memory: 11534336
COMET INFO: ----------------------------
COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


0.842


In [0]:
from comet_ml import Optimizer
optimizer = Optimizer("JmICwg0qOenLqzP7m5lXA1szo")

In [0]:

hidden_size = 100
params = """
n_epochs integer [1, 10] [1] log
learning_rate real [0.001, 0.005] [0.001] log
"""

# and report them to the Optimizer:
optimizer.set_params(params)


In [0]:
suggestion = optimizer.get_suggestion()

In [46]:
suggestion['n_epochs']

1

In [47]:
i = 0
while True:
  # 4. Get a suggestion
  try:
    suggestion = optimizer.get_suggestion()
    print ("Suggestion", suggestion)
  except comet_ml.NoMoreSuggestionsAvailable:
    # get_suggestion() will raise when no new suggestions
    # are available
    break

  
  # Build model, train, and get score:
  experiment = Experiment(api_key="JmICwg0qOenLqzP7m5lXA1szo", project_name="optimize2")
  
  print("iteration:", i)
  # In this example, we just sum up the values to optimize:
  score = train2score(n_epochs=suggestion['n_epochs'],
                      learning_rate=suggestion['learning_rate'], 
                      hidden_size=hidden_size)

  # 5. Report the score back, maximizes:
  suggestion.report_score("accuracy", score)
  i += 1


COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.ml/anaderi/optimize2/d0f324e3ced54d69a48d78b9c6524b86
COMET INFO:   Metrics:
COMET INFO:         sys.gpu.0.free_memory: 11985420288
COMET INFO:     sys.gpu.0.gpu_utilization: 0
COMET INFO:        sys.gpu.0.total_memory: 11996954624
COMET INFO:         sys.gpu.0.used_memory: 11534336
COMET INFO: ----------------------------


Suggestion <comet_ml.optimization.Suggestion object at 0x7fbd26a7b5c0>


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/anaderi/optimize2/c70b2722b29a47f28826acba86afecf2



iteration: 0


COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.ml/anaderi/optimize2/c70b2722b29a47f28826acba86afecf2
COMET INFO:   Metrics:
COMET INFO:                      accuracy: 0.8461
COMET INFO:                          loss: 0.4417598247528076
COMET INFO:         sys.gpu.0.free_memory: 11985420288
COMET INFO:     sys.gpu.0.gpu_utilization: 0
COMET INFO:        sys.gpu.0.total_memory: 11996954624
COMET INFO:         sys.gpu.0.used_memory: 11534336
COMET INFO: ----------------------------
COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


...

COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.ml/anaderi/optimize2/9facb3efc18b4254b252c20851678b9a
COMET INFO:   Metrics:
COMET INFO:                      accuracy: 0.8483
COMET INFO:                          loss: 0.4011300206184387
COMET INFO:         sys.gpu.0.free_memory: 11985420288
COMET INFO:     sys.gpu.0.gpu_utilization: 0
COMET INFO:        sys.gpu.0.total_memory: 11996954624
COMET INFO:         sys.gpu.0.used_memory: 11534336
COMET INFO: ----------------------------
COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


Suggestion <comet_ml.optimization.Suggestion object at 0x7fbd26a338d0>


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/anaderi/optimize2/bb76f6b59bd747b0a32559cb50f4ebfe



iteration: 20


KeyboardInterrupt: ignored

Use it to calculate the accuracy score here. How much did you get?

In [0]:
from torch import nn
device = torch.device('cuda')

### Insert missing pieces

model = nn.Sequential(
  Reshape(-1, 1, 28, ...),
  nn.Conv2d(in_channels=..., out_channels=10, kernel_size=3),
  nn.Dropout(p = ...),
  nn.MaxPool2d(2),
  Reshape(-1,...),
  nn.Linear(in_features=..., out_features=100),
  nn.ReLU(),
  nn.Linear(in_features=100, out_features=10)
).to(device)

In [0]:
# Define the hyperparameters of our model:
learning_rate = 0.005
batch_size = 64
dropout_rate = 0.2


# Creating the instances of the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

logger = Logger()

# Let's train for 20 epochs
for i_epoch in range(20):
  model.train() # setting the model to training mode
  for batch_X, batch_y in get_batches(X_train, y_train,
                                      batch_size=batch_size, shuffle=True):
    predictions = model(batch_X) # compute the predictions
    loss = loss_function(predictions, batch_y) # compute the loss
    logger.fill_train(loss.item())

    model.zero_grad() # zero the gradients
    loss.backward() # compute new gradients
    optimizer.step() # do an optimization step

  # Now, let's evaluate on the test part:
  model.eval() # setting the model to evaluatioin mode
  for batch_X, batch_y in get_batches(X_test, y_test,
                                      batch_size=batch_size):
    loss = loss_function(model(batch_X), batch_y)
    logger.fill_test(loss.item())
  
  logger.finish_epoch()

epoch #1 	 train_loss: 0.29409926 	 test_loss: 0.30546654


KeyboardInterrupt: ignored

In [0]:
n_parameters = 0
for parameter in model.parameters():
  n_parameters += parameter.reshape(-1).shape[0]

print(n_parameters)

170210


In [0]:
print(accuracy_score(y_test, get_test_predictions(model)))

0.8821


### Adding more convolutional magic

```
model = torch.nn.Sequential(
            Reshape(-1, 1, 28, 28),
            torch.nn.Conv2d(in_channels=1,
                            out_channels=8,
                            kernel_size=3,
                            padding=1),
            torch.nn.Dropout(p=dropout_rate),
            torch.nn.ELU(),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(in_channels=8,
                            out_channels=16,
                            kernel_size=3,
                            padding=1),
            torch.nn.Dropout(p=dropout_rate),
            torch.nn.ELU(),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(in_channels=16,
                            out_channels=32,
                            kernel_size=3,
                            padding=1),
            torch.nn.Dropout(p=dropout_rate),
            torch.nn.ELU(),
            torch.nn.MaxPool2d(2, padding=1),
            Reshape(-1, 512),
            torch.nn.Linear(512, 10)
        ).to(device)
```
